# Notebook 2_a

In [1]:
import pickle
from pathlib import Path
import tdt
import pandas as pd 
import os

In [2]:
import Notebook_2_helper_functions as Nb_2

In [3]:
mouse = 'T240'
data_day = '2019_09_27__T240'
date = '2019_09_27'

photo_day = 'T240-145905'

HowManyBack=1 

data_dir_input = "/Users/gilmandelbaum/Desktop/T240_mouse"
data_dir_output = "/Users/gilmandelbaum/Desktop/output/"

seq_str = "0a1b2a3b4ab5a6abc10a"

# Import behavior related data per state transition

In [4]:
behavior_data_states = pd.read_csv(data_dir_input+"/"+mouse+"/"+data_day+"/"+mouse+"_dataAnalog.csv")

In [5]:
behavior_data_states = Nb_2.prepare_behavior_data(behavior_data_states)

# Import photometry data


In [6]:
photometry_data = tdt.read_block(data_dir_input+"/"+mouse+"/"+photo_day)

read from t=0s to t=1345.46s


In [11]:
photometry_data.streams.grn1.data

gL1r	[struct]
bL1g	[struct]
gL2r	[struct]
gL2g	[struct]
bL2g	[struct]
red1	[struct]
grn1	[struct]
toGG	[struct]
Fi1r	[struct]
red2	[struct]
grn2	[struct]
froG	[struct]
Fi2r	[struct]

# Grab only what is needed from the TDT photo tanks and make a df. 

In [16]:
#The photometry TDT data structure has a lot of info. 
#We will create 1 data frame that has: 
# 
#['d2 R' , 'd1 R', 'd2 L', "d1 L", "toBehSys", "fromBehSys" ]

def reading_in_photometryData_all_Channels(photometry_data):
    photometry_all_channels_d = {}
    photometry_all_channels_d["d2 R"]=photometry_data.streams.grn1.data 
    photometry_all_channels_d["d1 R"]=photometry_data.streams.red1.data 
    
    
    photometry_all_channels_d["d2 L"]=photometry_data.streams.grn2.data 
    photometry_all_channels_d["d1 L"]=photometry_data.streams.red2.data 
    
    
    photometry_all_channels_d["toBehSys"]=photometry_data.streams.toGG.data 
    photometry_all_channels_d["fromBehSys"]=photometry_data.streams.froG.data 
    photometry_all_channels= pd.DataFrame(data=photometry_all_channels_d)
    
    return (photometry_all_channels)

In [17]:
photometry_all_channels=reading_in_photometryData_all_Channels(photometry_data)

# Clean the photometry data (or any neural data)

In [ ]:
photometry_all_channels_clean = Nb_2.handshake_behav_recording_sys(photometry_all_channels)

# Clean behavior states data 

In [ ]:
behavior_data_states_clean = Nb_2.define_behavior_data_states(behavior_data_states)

In [ ]:
photometry_all_channels_clean.head()

In [ ]:
cache_dir_plt = os.path.join(data_dir_output+'/'+mouse+'/'+data_day, str(HowManyBack)+'_Back',"0.plots")
path_to_plot = cache_dir_plt+"/"+seq_str[:seq_str.index("3")]+"_plot"

In [ ]:
fig.savefig(path_to_plot+"_1"+'.pdf')

# pre syncing steps for photometry data (or any neural data) and behavior data 

In [ ]:
behavior_data_states_clean_binNumberlist=Nb_2.number_of_bins_beh_data_per_trial(behavior_data_states_clean)

In [ ]:
photometry_all_channels_clean_binNumberlist = Nb_2.number_of_bins_photo_data_per_trial(photometry_all_channels_clean)

# Resample the photometry data (or any neural data) and align to 200 hz state transition behavioral data 

In [ ]:
aligned_behav_photo = Nb_2.resample_photoData_and_align_photo_with_beh(behavior_data_states_clean_binNumberlist,
                                                            behavior_data_states_clean,
                                                            photometry_all_channels_clean,
                                                            photometry_all_channels_clean_binNumberlist)

In [ ]:
aligned_behav_photo = Nb_2.tag_real_full_trial(aligned_behav_photo)

In [ ]:
root = Path(data_dir_output+"/"+mouse+"/"+data_day+'/'+str(HowManyBack)+"_Back")
d = mouse+"_"+date+"Notebook_2_a.pickle"
my_path = root / d 
my_file = open(my_path, 'wb')
my_file = pickle.dump(aligned_behav_photo,my_file)